# Parsing URLs with data from PostgreSQL dump file

Input dataframe:
 - view_id
 - website_id
 - session_id
 - created_at
 - url
 - referrer

Output dataframe:
- industries (this will contain a list of all the values in this field)
- course_of_study
- organisaton
- school

Summary of insights gained: (if any)

Written by: Howard and Jolene (only mostly optimizations)

In [3]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse, parse_qs, unquote

original_df = pd.read_csv('v1_pageview')
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113589 entries, 0 to 113588
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   view_id     113589 non-null  int64 
 1   website_id  113589 non-null  int64 
 2   session_id  113589 non-null  int64 
 3   created_at  113589 non-null  object
 4   url         113589 non-null  object
 5   referrer    99529 non-null   object
dtypes: int64(3), object(3)
memory usage: 5.2+ MB


##### Helper Functions

In [4]:
def extract_query_params(url):
    url = unquote(url) # make it human readable, not percentages
    query_params = parse_qs(urlparse(url).query)
    return query_params

Explaining process_query_params

What does the url look like when navigating the page?
```
// filter by industry Information and Communications Technology
filters[0][field]=industries&filters[0][values][0]=Information and Communications Technology

// filter by organization Google and SAP
filters[0][field]=organisation&filters[0][values][0]=SAP
filters[1][field]=organisation&filters[1][values][0]=Google

// filter by school 
filters[1][field]=school&filters[1][values][0]=National University of Singapore
filters[1][type]=any # we will ignore this part, not sure what it as, its always in 'any'
filters[2][field]=course_of_study
filters[2][values][0]=Economics%2C Psychology
filters[2][type]=any
```

To summarize, the number in filters[0][field] is the first/second/third filter applied etc. while the second value which is either 'field', 'value' or 'type' shows what the text after = is. `filters[1][field]=school&filters[1][values][0]=National University of Singapore` this means the second filter applied is a school filter, the value of the school filter applied is `filters[1][values][0]=National University of Singapore`.

So we can make use of this to extract the filters applied to each vistor URL.

In [5]:
def process_query_params(params):
    '''
    Parameters:
    params -> {'size': ['n_20_n'], 'filters[0][field]': ['industries'], 'filters[0][values][0]': ['Information and Communications Technology'], 'filters[0][type]': ['all']}
    
    Returns:
    {
        "search_query": "search term",
        "filter_name": ["filter value 1", "filter value 2"]
    }

    Note:
    - The filter_name is the name of the filter, e.g. industries, school etc.
    - size and type are ignored
    '''
    result = {}
    current_field = ''

    for key, value in params.items():
        if 'filters' in key:
            parts = key.split('[')
            field_or_value = parts[2].strip(']')

            if field_or_value == 'field':
                # if its a field then use it as a key
                current_field = value[0]
                result[current_field] = []
            elif field_or_value == 'type':
                # there's a type of all in all queries, not sure what that is and whether its relevant
                pass
            else:
                # if its a value then add it to the list by using the last saved field
                result[current_field].extend(value)
        elif key == "q":
            result["search_query"] = value[0]

    result = dict(result)
    return result

##### Main

Process URLs to extract query params

In [6]:
df = original_df.copy(deep=True)
df['url'] = df['url'].astype(str)
df['query_params'] = df['url'].apply(extract_query_params) # gets all the query params and makes it a dictionary 

In [ ]:
# x = pd.concat([new_df, new_df['query_params'].apply(lambda x: pd.Series(x, dtype="object"))], axis=1) #separate each of the key:value pairs into it's own column 

# x2 = x.assign(industries=lambda x: np.nan, 
#              school=lambda x: np.nan,
#              course_of_study=lambda x: np.nan,
#              organisation=lambda x: np.nan)  #create copy with the columns that i want

# def isnotNaN(num):
#     return num == num

# columns = ['view_id', 'website_id', 'session_id', 'created_at', 'url', 'referrer', 'query_params',
#            'q', 'size', 'current', 'sort-field', 'sort-direction', '_sm_au_', 'v', 'fbclid', 'trk',
#            'amp;amp;size', 'industries', 'school', 'course_of_study', 'organisation'] #columns that i eventually want


# for i in range(len(x2)): #go through original DF

#     row = x2.iloc[i] # for each row,

#     temp ={0:[None,list()], #theres 4 possible fields [field, list of values]
#            1:[None,list()],
#            2:[None,list()],
#            3:[None,list()]
#           }

#     for j in range(4): #identify which field corresponds to reach index
#         if isnotNaN(row.loc[f'filters[{j}][field]']):
#             temp[j][0] = row.loc[f'filters[{j}][field]'][0]
#         else:
#             break

#     for k in range(4): #condense all the values for each of the fields into list of values
#         for l in range(20):
#             if f'filters[{k}][values][{l}]' in row.index and isnotNaN(row.loc[f'filters[{k}][values][{l}]']):
#                 temp[k][1].append(row.loc[f'filters[{k}][values][{l}]'][0]) 
#             else:
#                 break

#     for z in range(4): #add the new column:values to dataframe
#         field = temp[z][0]
#         values = temp[z][1]
#         if field is None:
#             break
#         else:
#             x2.loc[i,field] = str(values)

In [7]:
# optimized version of the code above
df_processed = df.copy(deep=True)
df_processed['query_params'] = df_processed['query_params'].apply(process_query_params) # use only 1 for loop
df_processed = pd.DataFrame(df_processed['query_params'].values.tolist())

Why are there so many additional columns?

In [125]:
# ['i', 'ind', 'cou', 'indust', 'o', 'sch', 'course_of']
print(df_processed['indust'].value_counts())
print(df_processed['ind'].value_counts())
print(df_processed['i'].value_counts())
print(df_processed['cou'].value_counts())
print(df_processed['sch'].value_counts())
print(df_processed['o'].value_counts())
print(df_processed['course_of'].value_counts())
print(df_processed['wave_id'].value_counts())

indust
[]    4
Name: count, dtype: int64
ind
[]    22
Name: count, dtype: int64
i
[]    3
Name: count, dtype: int64
cou
[]    3
Name: count, dtype: int64
sch
[]    1
Name: count, dtype: int64
o
[]    1
Name: count, dtype: int64
course_of
[]    1
Name: count, dtype: int64
wave_id
[n_2_n]           8
[n_1_n]           7
[n_3_n]           5
[n_0_n]           2
[n_0_n, n_2_n]    1
[n_3_n, n_1_n]    1
Name: count, dtype: int64


We see that the additional columns all contain empty list, so we can safely drop them. As for wave_id, we will leave it in for now.

In [126]:
# drop unused columns ['i', 'ind', 'cou', 'indust', 'o', 'sch', 'course_of']
df_processed = df_processed.drop(['i', 'ind', 'cou', 'indust', 'o', 'sch', 'course_of'], axis=1)
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113589 entries, 0 to 113588
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   search_query     20959 non-null  object
 1   industries       29829 non-null  object
 2   course_of_study  7103 non-null   object
 3   organisation     13074 non-null  object
 4   school           3657 non-null   object
 5   course           1 non-null      object
 6   wave_id          24 non-null     object
dtypes: object(7)
memory usage: 6.1+ MB


Export to CSV

In [18]:
df_processed.to_csv('data-preprocessed.csv', index=False)